# Курсовая работа № 2
## Аналитика. Начальный уровень

Курсовая работа состоит из двух частей – обязательной и дополнительной. **Для зачета необходимо выполнение только первой части.** Выполнение второй части может потребовать дополнительные знания Python.

- [Часть первая](#Часть-первая)
- [Часть вторая](#Часть-вторая)

## Часть первая


Перед вами стоит задача – подготовить аналитический отчет для HR-отдела. На основании проведенной аналитики предполагается составить рекомендации для отдела кадров по стратегии набора персонала, а также по взаимодействию с уже имеющимися сотрудниками.
<br><br> В базе данных лежит набор таблиц, которые содержат данные о сотрудниках вымышленной компании.
Сделайте обзор штата сотрудников компании. Составьте набор предметов исследования, а затем проверьте их на данных. Вся аналитика должна быть выполена с помощью SQL. Впоследствии данные можно визуализировать, однако финальные датафреймы для графиков также должны быть подготовлены с помощью SQL. <br><br>

Примеры гипотез:
1. Есть зависимость между `perfomance score` и тем, под чьим руководством работает сотрудник.
2. Есть зависимость между продолжительностью работы в компании и семейным положением сотрудника.
2. Есть зависимость между продолжительностью работы в компании и возрастом сотрудника.

<br><br>
Параметры для подключения следующие: хост – `dsstudents.skillbox.ru`, порт – `5432`, имя базы данных – `human_resources`, пользователь – `readonly`, пароль – `6hajV34RTQfmxhS`. Таблицы, доступные для анализа, – `hr_dataset`, `production_staff`, `recruting_costs`, `salary_grid`.

In [1]:
### YOUR CODE HERE ###

import psycopg2
import pandas as pd
import numpy as np
import sqlite3

In [295]:
# пять запросов ниже не нужны для аналитики - только чтобы я мог посмотреть, что делается в sql-таблицах

sql_str = []
columns_list = [[], [], [], [], []]

columns = ('marriedid', 'maritalstatusid', 'genderid', 'empstatus_id', 'deptid', 'state', 'sex', \
           'citizendesc', '"Hispanic/Latino"', 'racedesc', 'department', '"Manager Name"', '"Employee Source"')

sql_str.append("SELECT * FROM hr_dataset;")
sql_str.append("SELECT * FROM production_staff;")
sql_str.append("SELECT * FROM recruiting_costs;")
sql_str.append("SELECT * FROM salary_grid;")
sql_str.append("SELECT * FROM information_schema.columns WHERE table_schema='public';")

# Для аналитики

for i in columns:

    sql_str.append('SELECT ' + i + ', count("Days Employed"), avg("Days Employed") \
                    FROM hr_dataset \
                    WHERE "Date of Termination" is not null GROUP BY ' + i + ';')
    columns_list.append([i, 'count of workers', 'average Days Employed'])

    
    sql_str.append('SELECT que1."Performance", que1.' + i + ', que1."quantity", que2."sum_quantity", CAST(que1."quantity" AS float) / que2."sum_quantity" \
                   FROM (SELECT \
                    CASE \
                       WHEN "Performance Score" IN (\'Exceeds\', \'Exceptional\', \'Fully Meets\') \
                       THEN \'Good performance\' \
                       WHEN "Performance Score" = \'Needs Improvement\' \
                       THEN \'No good performance\' \
                       ELSE \'Others\' \
                   END AS "Performance", \
                   ' + i + ', count(*) AS quantity \
                   FROM hr_dataset \
                   GROUP BY "Performance", ' + i + ') que1 \
                INNER JOIN \
                   (SELECT ' + i +', count(*) as sum_quantity \
                   FROM hr_dataset \
                   GROUP BY ' + i + ') que2 \
               ON que1.' + i + ' = que2.' + i + ' \
               ORDER BY "Performance", ' + i)
    columns_list.append(['Performance', i, 'quantity of workers per group', 'workers in group ' + i, "share of workers"])

# Для проверки тождественности полей
    
sql_str.append('SELECT count(*) FROM \
(SELECT \
    CASE \
        WHEN maritalstatusid = 0 \
        THEN \'Single\' \
        WHEN maritalstatusid = 1 \
        THEN \'Married\' \
        WHEN maritalstatusid = 2 \
        THEN \'Divorced\' \
        WHEN maritalstatusid = 3 \
        THEN \'Separated\' \
        WHEN maritalstatusid = 4 \
        THEN \'Widowed\' \
    END as marital, maritaldesc \
FROM hr_dataset) AS que \
WHERE marital <> maritaldesc')
columns_list.append(['count of maritalstatusid <> maritaldesc'])

sql_str.append('SELECT count(*) FROM \
(SELECT \
    CASE \
        WHEN genderid = 0 \
        THEN \'Female\' \
        WHEN genderid = 1 \
        THEN \'Male\' \
    END as gender, sex \
FROM hr_dataset) AS que \
WHERE gender <> sex')
columns_list.append(['genderid <> sex'])

sql_str.append('SELECT count(*) FROM \
(SELECT \
    CASE \
        WHEN empstatus_id = 1 \
        THEN \'Active\' \
        WHEN empstatus_id = 2 \
        THEN \'Future Start\' \
        WHEN empstatus_id = 3 \
        THEN \'Leave of Absence\' \
        WHEN empstatus_id = 4 \
        THEN \'Terminated for Cause\' \
        WHEN empstatus_id = 5 \
        THEN \'Voluntarily Terminated\' \
    END as empstatus, "Employment Status" \
FROM hr_dataset) AS que \
WHERE empstatus <> "Employment Status"')
columns_list.append(['empstatus_id <> Employment Status'])

In [296]:
# Выполним запросы и заберем данные из таблиц БД

pg_list = []

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "human_resources",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)

cursor = conn.cursor()

for i in sql_str:
    cursor.execute(i)
    pg_list.append([a for a in cursor.fetchall()])

conn.commit()

In [188]:
# служебный блок, для аналитики не нужен - только для моего удобства

info_df = pd.DataFrame(pg_list[4])

hr_df = pd.DataFrame(pg_list[0], columns = info_df.loc[info_df[2] == 'hr_dataset'][3].tolist(), \
                    index = np.array(pg_list[0])[:,0])
hr_df = hr_df.drop('id', axis=1)
hr_df.dob = pd.to_datetime(hr_df.dob, errors = 'coerce')
hr_df['Date of Hire'] = pd.to_datetime(hr_df['Date of Hire'], errors = 'coerce')
hr_df['Date of Termination'] = pd.to_datetime(hr_df['Date of Termination'], errors = 'coerce')


production_df = pd.DataFrame(pg_list[1], columns = info_df.loc[info_df[2] == 'production_staff'][3].tolist(), \
                            index = np.array(pg_list[1])[:,0])
production_df = production_df.drop('id', axis=1)
production_df['Date of Hire'] = pd.to_datetime(production_df['Date of Hire'], errors = 'coerce')
production_df['TermDate'] = pd.to_datetime(production_df['TermDate'], errors = 'coerce')


recruit_df = pd.DataFrame(pg_list[2], columns = info_df.loc[info_df[2] == 'recruiting_costs'][3].tolist(), \
                         index = np.array(pg_list[2])[:,0])
recruit_df = recruit_df.drop('id', axis=1)


salary_df = pd.DataFrame(pg_list[3], columns = info_df.loc[info_df[2] == 'salary_grid'][3].tolist(), \
                        index = np.array(pg_list[3])[:,0])
salary_df = salary_df.drop('id', axis=1)

pd.options.display.max_columns = 40

Предположим, что hr-отделу нужно:
1. Исследовать параметр Days Employed среди уволенных сотрудников (для неуволенных мы не можем знать, сколько времени они проработают).
2. Увеличить Performance Score, набирая классы работников, которые ранее показали большое значение этого показателя.


Примем, что Performance Score у эффективных сотрудников = ['Exceeds', 'Exceptional', 'Fully Meets'], а у неэффективных = 'Needs Improvement', остальные непонятные характеристики - в отдельную группу Others.

1. marriedid и Days Employed

In [189]:
pd.DataFrame(pg_list[5], columns = columns_list[5])

,marriedid,count of workers,average Days Employed
0,0,57,764.7368421052631579
1,1,46,767.2173913043478261


В среднем работники с marriedid = 1 работают в компании чуть дольше, чем неженатые.

2. marriedid и Performance Score

In [282]:
pd.DataFrame(pg_list[6], columns = columns_list[6])

,Performance,marriedid,quantity of workers per group,workers in group marriedid,share of workers
0,Good performance,0,134,187,0.716578
1,Good performance,1,84,123,0.682927
2,No good performance,0,9,187,0.048128
3,No good performance,1,6,123,0.048780
4,Others,0,44,187,0.235294
5,Others,1,33,123,0.268293


Доля работников с marriedid = 0 с высокой производительностью = 0.7165778  
Доля работников с marriedid = 1 с высокой производительностью = 0.6829267  
По работникам с плохой производительностью данных мало - менее 10 случаев на группу

3. maritalstatusid и Days Employed

In [192]:
pd.DataFrame(pg_list[7], columns = columns_list[7])

,maritalstatusid,count of workers,average Days Employed
0,0,36,879.1111111111111111
1,1,46,767.2173913043478261
2,3,1,1675.0000000000000000
3,4,4,414.5000000000000000
4,2,16,538.0625000000000000


Не корректно делать выводы по сотрудникам с maritalstatusid = 3 и 4, т.к. мало статистики собрано.
Из остальных предпочтительной группой является maritalstatusid = 0, следующая группа с maritalstatusid = 1 чуть хуже.

4. maritalstatusid и Performance Score

In [194]:
pd.DataFrame(pg_list[8], columns = columns_list[8])

,Performance,maritalstatusid,quantity of workers per group,workers withmaritalstatusid,share of workers
0,Good performance,0,99,137,0.722628
1,Good performance,1,84,123,0.682927
2,Good performance,2,21,30,0.700000
3,Good performance,3,10,12,0.833333
4,Good performance,4,4,8,0.500000
5,No good performance,0,7,137,0.051095
6,No good performance,1,6,123,0.048780
7,No good performance,2,2,30,0.066667
8,Others,0,31,137,0.226277
9,Others,1,33,123,0.268293


Доля работников с maritalstatusid = 0 с высокой производительностью = 0.722628  
Доля работников с maritalstatusid = 1 с высокой производительностью = 0.682927  
Доля работников с maritalstatusid = 2 с высокой производительностью = 0.7  
Доля работников с maritalstatusid = 3 с высокой производительностью = 0.833333  

Группу с maritalstatusid = 4 исключил, тк мало экспериментов. Лучшая оказалась группа maritalstatusid = 3, но лучше получить больше данных.

5. genderid и Days Employed

In [195]:
pd.DataFrame(pg_list[9], columns = columns_list[9])

,genderid,count of workers,average Days Employed
0,0,60,750.8000000000000000
1,1,43,786.8372093023255814


Работники с genderid = 0 (женщины) в среднем работают дольше, чем работники с genderid = 1 (мужчины).

6. genderid и Performance Score

In [196]:
pd.DataFrame(pg_list[10], columns = columns_list[10])

,Performance,genderid,quantity of workers per group,workers withgenderid,share of workers
0,Good performance,0,122,177,0.689266
1,Good performance,1,96,133,0.721805
2,No good performance,0,5,177,0.028249
3,No good performance,1,10,133,0.075188
4,Others,0,50,177,0.282486
5,Others,1,27,133,0.203008


Доля работников с genderid = 0 (женщины) с высокой производительностью = 0.6892655367231638  
Доля работников с genderid = 1 (мужчины) с высокой производительностью = 0.7218045112781954

7. empstatus_id и Days Employed

In [299]:
pd.DataFrame(pg_list[11], columns = columns_list[11])

,empstatus_id,count of workers,average Days Employed
0,1,1,58.0000000000000000
1,5,88,781.9090909090909091
2,4,14,715.4285714285714286


Вошли не все значения, т.к. в выборку включены только сотрудники с Date of Termination, не равным нулю, т.е. работающие и отпускники не попали в таблицу.
В выборку почем

8. empstatus_id и Performance Score

In [199]:
pd.DataFrame(pg_list[12], columns = columns_list[12])

,Performance,empstatus_id,quantity of workers per group,workers withempstatus_id,share of workers
0,Good performance,1,140,183,0.765027
1,Good performance,2,2,11,0.181818
2,Good performance,3,12,14,0.857143
3,Good performance,4,6,14,0.428571
4,Good performance,5,58,88,0.659091
5,No good performance,1,7,183,0.038251
6,No good performance,4,4,14,0.285714
7,No good performance,5,4,88,0.045455
8,Others,1,36,183,0.196721
9,Others,2,9,11,0.818182


Не понял, что значит

Доля работников с empstatus_id = 1 с высокой производительностью = 0.765027  
Доля работников с empstatus_id = 1 с высокой производительностью = 0.181818  
Доля работников с empstatus_id = 4 с высокой производительностью = 0.6590909090909091

In [286]:
hr_df.head()

,Employee Name,Employee Number,marriedid,maritalstatusid,genderid,empstatus_id,deptid,perf_scoreid,age,Pay Rate,state,zip,dob,sex,maritaldesc,citizendesc,Hispanic/Latino,racedesc,Date of Hire,Days Employed,Date of Termination,Reason For Term,Employment Status,department,position,Manager Name,Employee Source,Performance Score
1,"Brown, Mia",1103024456,1,1,0,1,1,3,30,28.50,MA,1450,1987-11-24,Female,Married,US Citizen,No,Black or African American,2008-10-27,3317,NaT,N/A - still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets
2,"LaRotonda, William",1106026572,0,2,1,1,1,3,34,23.00,MA,1460,1984-04-26,Male,Divorced,US Citizen,No,Black or African American,2014-01-06,1420,NaT,N/A - still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Website Banner Ads,Fully Meets
3,"Steans, Tyrone",1302053333,0,0,1,1,1,3,31,29.00,MA,2703,1986-09-01,Male,Single,US Citizen,No,White,2014-09-29,1154,NaT,N/A - still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Internet Search,Fully Meets
4,"Howard, Estelle",1211050782,1,1,0,1,1,9,32,21.50,MA,2170,1985-09-16,Female,Married,US Citizen,No,White,2015-02-16,58,2015-04-15,N/A - still employed,Active,Admin Offices,Administrative Assistant,Brandon R. LeBlanc,Pay Per Click - Google,N/A- too early to review
5,"Singh, Nan",1307059817,0,0,0,1,1,9,30,16.56,MA,2330,1988-05-19,Female,Single,US Citizen,No,White,2015-05-01,940,NaT,N/A - still employed,Active,Admin Offices,Administrative Assistant,Brandon R. LeBlanc,Website Banner Ads,N/A- too early to review


In [302]:
hr_df.query('empstatus_id == [1,5,4]')[['empstatus_id', 'Employment Status']].head(50)

,empstatus_id,Employment Status
1,1,Active
2,1,Active
3,1,Active
4,1,Active
5,1,Active
6,5,Voluntarily Terminated
7,1,Active
8,5,Voluntarily Terminated
9,1,Active
10,1,Active


In [300]:
hr_df.query('empstatus_id == [0, 2,3]')[['empstatus_id', 'Employment Status']]

,empstatus_id,Employment Status
17,3,Leave of Absence
45,2,Future Start
48,3,Leave of Absence
49,3,Leave of Absence
50,2,Future Start
73,3,Leave of Absence
79,3,Leave of Absence
84,3,Leave of Absence
91,2,Future Start
95,2,Future Start


In [293]:
hr_df[['Employment Status', 'empstatus_id']].head(50)

,Employment Status,empstatus_id
1,Active,1
2,Active,1
3,Active,1
4,Active,1
5,Active,1
6,Voluntarily Terminated,5
7,Active,1
8,Voluntarily Terminated,5
9,Active,1
10,Active,1


In [290]:
hr_df.empstatus_id.unique()

array([1, 5, 4, 3, 2])

9. deptid и Days Employed

In [201]:
pd.DataFrame(pg_list[13], columns = columns_list[13])

,deptid,count of workers,average Days Employed
0,6,4,935.2500000000000000
1,1,3,808.0000000000000000
2,3,10,428.8000000000000000
3,5,83,793.2891566265060241
4,4,3,862.0000000000000000


10. deptid и Performance Score

In [202]:
pd.DataFrame(pg_list[14], columns = columns_list[14])

,Performance,deptid,quantity of workers per group,workers withdeptid,share of workers
0,Good performance,1,7,10,0.700000
1,Good performance,2,1,1,1.000000
2,Good performance,3,32,50,0.640000
3,Good performance,4,7,10,0.700000
4,Good performance,5,147,208,0.706731
5,Good performance,6,24,31,0.774194
6,No good performance,3,1,50,0.020000
7,No good performance,4,1,10,0.100000
8,No good performance,5,11,208,0.052885
9,No good performance,6,2,31,0.064516


11. state и Days Employed

In [203]:
pd.DataFrame(pg_list[15], columns = columns_list[15])

,state,count of workers,average Days Employed
0,PA,1,899.0000000000000000
1,TN,1,1334.0000000000000000
2,OH,1,425.0000000000000000
3,MA,98,759.2755102040816327
4,VA,1,1083.0000000000000000
5,CT,1,732.0000000000000000


12. deptid и Performance Score

In [204]:
pd.DataFrame(pg_list[16], columns = columns_list[16])

,Performance,state,quantity of workers per group,workers withstate,share of workers
0,Good performance,AZ,1,1,1.000000
1,Good performance,CA,1,1,1.000000
2,Good performance,CO,1,1,1.000000
3,Good performance,CT,5,6,0.833333
4,Good performance,FL,1,1,1.000000
5,Good performance,GA,1,1,1.000000
6,Good performance,ID,1,1,1.000000
7,Good performance,KY,1,1,1.000000
8,Good performance,MA,190,275,0.690909
9,Good performance,ME,1,1,1.000000


13. state и Days Employed

In [205]:
pd.DataFrame(pg_list[17], columns = columns_list[17])

,sex,count of workers,average Days Employed
0,Female,60,750.8000000000000000
1,Male,43,786.8372093023255814


14. deptid и Performance Score

In [206]:
pd.DataFrame(pg_list[18], columns = columns_list[18])

,Performance,sex,quantity of workers per group,workers withsex,share of workers
0,Good performance,Female,122,177,0.689266
1,Good performance,Male,96,133,0.721805
2,No good performance,Female,5,177,0.028249
3,No good performance,Male,10,133,0.075188
4,Others,Female,50,177,0.282486
5,Others,Male,27,133,0.203008


15. state и Days Employed

In [207]:
pd.DataFrame(pg_list[19], columns = columns_list[19])

,citizendesc,count of workers,average Days Employed
0,Eligible NonCitizen,5,867.8000000000000000
1,Non-Citizen,3,697.3333333333333333
2,US Citizen,95,762.6421052631578947


16. deptid и Performance Score

In [208]:
pd.DataFrame(pg_list[20], columns = columns_list[20])

,Performance,citizendesc,quantity of workers per group,workers withcitizendesc,share of workers
0,Good performance,Eligible NonCitizen,7,12,0.583333
1,Good performance,Non-Citizen,4,4,1.000000
2,Good performance,US Citizen,207,294,0.704082
3,No good performance,Eligible NonCitizen,1,12,0.083333
4,No good performance,US Citizen,14,294,0.047619
5,Others,Eligible NonCitizen,4,12,0.333333
6,Others,US Citizen,73,294,0.248299


17. state и Days Employed

In [209]:
pd.DataFrame(pg_list[21], columns = columns_list[21])

,"""Hispanic/Latino""",count of workers,average Days Employed
0,No,94,772.7234042553191489
1,Yes,9,694.0000000000000000


18. deptid и Performance Score

In [210]:
pd.DataFrame(pg_list[22], columns = columns_list[22])

,Performance,"""Hispanic/Latino""",quantity of workers per group,"workers with""Hispanic/Latino""",share of workers
0,Good performance,No,200,281,0.711744
1,Good performance,Yes,16,27,0.592593
2,Good performance,no,1,1,1.000000
3,Good performance,yes,1,1,1.000000
4,No good performance,No,14,281,0.049822
5,No good performance,Yes,1,27,0.037037
6,Others,No,67,281,0.238434
7,Others,Yes,10,27,0.370370


19. state и Days Employed

In [211]:
pd.DataFrame(pg_list[23], columns = columns_list[23])

,racedesc,count of workers,average Days Employed
0,Hispanic,1,2.0000000000000000
1,Two or more races,7,1100.5714285714285714
2,Asian,11,679.0000000000000000
3,White,67,742.5373134328358209
4,Black or African American,17,821.0000000000000000


20. deptid и Performance Score

In [212]:
pd.DataFrame(pg_list[24], columns = columns_list[24])

,Performance,racedesc,quantity of workers per group,workers withracedesc,share of workers
0,Good performance,American Indian or Alaska Native,4,4,1.000000
1,Good performance,Asian,24,34,0.705882
2,Good performance,Black or African American,42,57,0.736842
3,Good performance,Hispanic,2,4,0.500000
4,Good performance,Two or more races,13,18,0.722222
5,Good performance,White,133,193,0.689119
6,No good performance,Asian,1,34,0.029412
7,No good performance,Black or African American,7,57,0.122807
8,No good performance,Hispanic,1,4,0.250000
9,No good performance,Two or more races,1,18,0.055556


21. state и Days Employed

In [213]:
pd.DataFrame(pg_list[25], columns = columns_list[25])

,department,count of workers,average Days Employed
0,Admin Offices,3,808.0000000000000000
1,IT/IS,10,428.8000000000000000
2,Software Engineering,3,862.0000000000000000
3,Sales,4,935.2500000000000000
4,Production,83,793.2891566265060241


22. deptid и Performance Score

In [214]:
pd.DataFrame(pg_list[26], columns = columns_list[26])

,Performance,department,quantity of workers per group,workers withdepartment,share of workers
0,Good performance,Admin Offices,7,10,0.700000
1,Good performance,Executive Office,1,1,1.000000
2,Good performance,IT/IS,32,50,0.640000
3,Good performance,Production,147,208,0.706731
4,Good performance,Sales,24,31,0.774194
5,Good performance,Software Engineering,7,10,0.700000
6,No good performance,IT/IS,1,50,0.020000
7,No good performance,Production,11,208,0.052885
8,No good performance,Sales,2,31,0.064516
9,No good performance,Software Engineering,1,10,0.100000


23. state и Days Employed

In [215]:
pd.DataFrame(pg_list[27], columns = columns_list[27])

,"""Manager Name""",count of workers,average Days Employed
0,Michael Albert,9,653.3333333333333333
1,Simon Roup,8,345.7500000000000000
2,John Smith,3,947.3333333333333333
3,Amy Dunn,13,781.3076923076923077
4,Brannon Miller,6,1059.0000000000000000
5,Webster Butler,13,978.1538461538461538
6,Brandon R. LeBlanc,2,394.0000000000000000
7,Janet King,6,1075.8333333333333333
8,Elijiah Gray,8,926.5000000000000000
9,Alex Sweetwater,3,862.0000000000000000


24. deptid и Performance Score

In [216]:
pd.DataFrame(pg_list[28], columns = columns_list[28])

,Performance,"""Manager Name""",quantity of workers per group,"workers with""Manager Name""",share of workers
0,Good performance,Alex Sweetwater,6,9,0.666667
1,Good performance,Amy Dunn,14,21,0.666667
2,Good performance,Board of Directors,2,2,1.000000
3,Good performance,Brandon R. LeBlanc,4,7,0.571429
4,Good performance,Brannon Miller,15,21,0.714286
5,Good performance,Brian Champaigne,8,8,1.000000
6,Good performance,David Stanley,16,21,0.761905
7,Good performance,Debra Houlihan,2,3,0.666667
8,Good performance,Elijiah Gray,15,22,0.681818
9,Good performance,Eric Dougall,4,4,1.000000


25. state и Days Employed

In [218]:
pd.DataFrame(pg_list[29], columns = columns_list[29])

,"""Employee Source""",count of workers,average Days Employed
0,Company Intranet - Partner,1,444.0000000000000000
1,Professional Society,3,1202.3333333333333333
2,On-line Web application,1,194.0000000000000000
3,Billboard,5,577.8000000000000000
4,Pay Per Click,1,2.0000000000000000
5,MBTA ads,4,989.0000000000000000
6,Newspager/Magazine,5,482.6000000000000000
7,Internet Search,2,1057.5000000000000000
8,Search Engine - Google Bing Yahoo,15,828.0666666666666667
9,Pay Per Click - Google,4,419.5000000000000000


26. deptid и Performance Score

In [219]:
pd.DataFrame(pg_list[30], columns = columns_list[30])

,Performance,"""Employee Source""",quantity of workers per group,"workers with""Employee Source""",share of workers
0,Good performance,Billboard,12,16,0.750000
1,Good performance,Careerbuilder,1,1,1.000000
2,Good performance,Company Intranet - Partner,1,1,1.000000
3,Good performance,Diversity Job Fair,20,29,0.689655
4,Good performance,Employee Referral,21,31,0.677419
5,Good performance,Glassdoor,10,14,0.714286
6,Good performance,Indeed,8,8,1.000000
7,Good performance,Information Session,3,4,0.750000
8,Good performance,Internet Search,4,6,0.666667
9,Good performance,MBTA ads,14,17,0.823529


In [265]:
pd.DataFrame(pg_list[31]).shape

(1, 1)

In [269]:
pd.DataFrame(pg_list[31], columns = [columns_list[31]])

,count of maritalstatusid <> maritaldesc
0,0


In [279]:
pd.DataFrame(pg_list[32], columns = [columns_list[32]])

,genderid <> sex
0,0


In [297]:
pd.DataFrame(pg_list[33], columns = [columns_list[33]])

,empstatus_id <> Employment Status
0,0


In [23]:
hr_df.head()

,Employee Name,Employee Number,marriedid,maritalstatusid,genderid,empstatus_id,deptid,perf_scoreid,age,Pay Rate,state,zip,dob,sex,maritaldesc,citizendesc,Hispanic/Latino,racedesc,Date of Hire,Days Employed,Date of Termination,Reason For Term,Employment Status,department,position,Manager Name,Employee Source,Performance Score
1,"Brown, Mia",1103024456,1,1,0,1,1,3,30,28.50,MA,1450,1987-11-24,Female,Married,US Citizen,No,Black or African American,2008-10-27,3317,NaT,N/A - still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets
2,"LaRotonda, William",1106026572,0,2,1,1,1,3,34,23.00,MA,1460,1984-04-26,Male,Divorced,US Citizen,No,Black or African American,2014-01-06,1420,NaT,N/A - still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Website Banner Ads,Fully Meets
3,"Steans, Tyrone",1302053333,0,0,1,1,1,3,31,29.00,MA,2703,1986-09-01,Male,Single,US Citizen,No,White,2014-09-29,1154,NaT,N/A - still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Internet Search,Fully Meets
4,"Howard, Estelle",1211050782,1,1,0,1,1,9,32,21.50,MA,2170,1985-09-16,Female,Married,US Citizen,No,White,2015-02-16,58,2015-04-15,N/A - still employed,Active,Admin Offices,Administrative Assistant,Brandon R. LeBlanc,Pay Per Click - Google,N/A- too early to review
5,"Singh, Nan",1307059817,0,0,0,1,1,9,30,16.56,MA,2330,1988-05-19,Female,Single,US Citizen,No,White,2015-05-01,940,NaT,N/A - still employed,Active,Admin Offices,Administrative Assistant,Brandon R. LeBlanc,Website Banner Ads,N/A- too early to review


In [280]:
production_df

,Employee Name,Race Desc,Date of Hire,TermDate,Reason for Term,Employment Status,Department,Position,Pay,Manager Name,Performance Score,Abutments/Hour Wk 1,Abutments/Hour Wk 2,Daily Error Rate,90-day Complaints
1,"Albert, Michael",White,2011-08-01,NaT,N/A - still employed,Active,Production,Production Manager,$54.50,Elisa Bramante,Fully Meets,0.0,0.0,0.0,0.0
2,"Bozzi, Charles",Asian,2013-09-30,2014-08-07,retiring,Voluntarily Terminated,Production,Production Manager,$50.50,Elisa Bramante,Fully Meets,0.0,0.0,0.0,0.0
3,"Butler, Webster L",White,2016-01-28,NaT,N/A - still employed,Active,Production,Production Manager,$55.00,Elisa Bramante,Exceeds,0.0,0.0,0.0,0.0
4,"Dunn, Amy",White,2014-09-18,NaT,N/A - still employed,Active,Production,Production Manager,$51.00,Elisa Bramante,Fully Meets,0.0,0.0,0.0,0.0
5,"Gray, Elijiah",White,2015-06-02,NaT,N/A - still employed,Active,Production,Production Manager,$54.00,Elisa Bramante,Fully Meets,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,None,None,NaT,NaT,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN
253,None,None,NaT,NaT,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN
254,None,None,NaT,NaT,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN
255,None,None,NaT,NaT,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN


In [25]:
recruit_df

,Employment Source,January,February,March,April,May,June,July,August,September,October,November,December,Total
1,Billboard,520,520,520,520,0,0,612,612,729,749,910,500,6192
2,Careerbuilder,410,410,410,820,820,410,410,820,820,1230,820,410,7790
3,Company Intranet - Partner,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Diversity Job Fair,0,5129,0,0,0,0,0,4892,0,0,0,0,10021
5,Employee Referral,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Glassdoor,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Information Session,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Internet Search,0,0,0,0,0,0,0,0,0,0,0,0,0
9,MBTA ads,640,640,640,640,640,640,640,1300,1300,1300,1300,1300,10980
10,Monster.com,500,500,500,440,500,500,440,500,440,440,500,500,5760


In [26]:
salary_df

,Position,Salary Min,Salary Mid,Salary Max,Hourly Min,Hourly Mid,Hourly Max
1,Administrative Assistant,30000,40000,50000,14.42,19.23,24.04
2,Sr. Administrative Assistant,35000,45000,55000,16.83,21.63,26.44
3,Accountant I,42274,51425,62299,20.32,24.72,29.95
4,Accountant II,50490,62158,74658,24.27,29.88,35.89
5,Sr. Accountant,63264,76988,92454,30.42,37.01,44.45
6,Network Engineer,50845,66850,88279,24.44,32.14,42.44
7,Sr. Network Engineer,79428,99458,120451,38.19,47.82,57.91
8,Database Administrator,50569,68306,93312,24.31,32.84,44.86
9,Sr. DBA,92863,116007,139170,44.65,55.77,66.91
10,Production Technician I,30000,40000,50000,14.42,19.23,24.04


1. hr_dataset_list имеет два поля 'sex' и 'genderid', которые хранят один и тот же параметр, но в разной форме.

## Часть вторая

Перед вами стоит задача – подготовить аналитический ответ для SMM-отдела компании Skillbox. <br> Объектом анализа является  [паблик Skillbox Вконтакте](https://vk.com/skillbox_education). <br> <br> 
Подключитесь к  API VK и выгрузите посты со стены паблика Skillbox за интересующий период (определите самостоятельно и обоснуйте). Проанализируйте влияние различных факторов (например, времени публикации) на вовлеченность пользователей (количество лайков, комментариев, голосов в опросах). Сделайте аналитику по рубрикам (примеры рубрик: дизайн-битва, игра по управлению), которые есть в паблике. Выбрать нужные посты можно с помощью регулярных выражений. Составьте перечень рекомандаций для SMM-отдела по итогам анализа. <br> <br> 

Дополнительные инструкции по работе с API VK расположены [здесь](https://nbviewer.jupyter.org/github/vikaborel/dstutorials/blob/master/Дополнительные%20инструкции.ipynb).

In [ ]:
### YOUR CODE HERE ###


